# Ensemble Model

In [86]:
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators import H2ORandomForestEstimator, H2OGradientBoostingEstimator, H2OGeneralizedLinearEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
import pandas as pd

In [87]:
# Initialize H2O
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 17 mins
H2O_cluster_timezone:,Asia/Singapore
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,1 month and 25 days
H2O_cluster_name:,chunhong
H2O_cluster_total_nodes:,2
H2O_cluster_free_memory:,5.385 Gb
H2O_cluster_total_cores:,32
H2O_cluster_allowed_cores:,32
H2O_cluster_status:,"locked, healthy"


In [88]:
# Load the pre-split datasets into H2O
X_train = h2o.import_file("./dataset/X_train.csv")
y_train = h2o.import_file("./dataset/y_train.csv")
X_val = h2o.import_file("./dataset/X_val.csv")
y_val = h2o.import_file("./dataset/y_val.csv")
X_test = h2o.import_file("./dataset/X_test.csv")
y_test = h2o.import_file("./dataset/y_test.csv")

# Combine X and y for H2O training (H2O expects the target to be part of the dataset)
train = X_train.cbind(y_train)
val = X_val.cbind(y_val)
test = X_test.cbind(y_test)

# Define the target and features
target = "yearly_compensation"
features = [col for col in train.columns if col != target]

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


### Bagging

In [89]:
# Train a Random Forest (Bagging)
# Also one of the Base Models (Level 0 Models)
rf_model = H2ORandomForestEstimator(
    ntrees=100,          # Number of trees
    max_depth=20,        # Maximum depth of each tree
    min_rows=5,          # Minimum number of rows per leaf
    sample_rate=0.8,     # Row sampling rate per tree
    seed=42,             # Random seed for reproducibility
    nfolds=5,
    keep_cross_validation_predictions=True
)

# Train the model
rf_model.train(x=features, y=target, training_frame=train, validation_frame=val)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_model_python_1735308900165_63


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    100                100                         843356                 18           20           19.78         636           695           667.3

ModelMetricsRegression: drf
** Reported on train data. **

MSE: 10.172963741851962
RMSE: 3.189508385606152
MAE: 2.467496793161229
RMSLE: 0.5757411672255234
Mean Residual Deviance: 10.172963741851962

ModelMetricsRegression: drf
** Reported on validation data. **

MSE: 9.823978913450421
RMSE: 3.134322720054593
MAE: 2.449714837981213
RMSLE: 0.5798700287783111
Mean Residual Deviance: 9.823978913450421

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 10.302086780001131
RMSE: 3.209686399011768
MAE: 2.5335557640341007
RMSLE: 0.5920549827911843
Mean Residual Deviance: 10.302086780001131

Cross-Validation Metrics Summary: 
                        mean      sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  --------  ----------  ------------  ------------  ------------  ------------  ------------
aic                     nan       0           nan           nan           nan           nan           nan
loglikelihood           nan       0           nan           nan           nan           nan           nan
mae                     2.53414   0.0510449   2.56571       2.51034       2.55654       2.45619       2.5819
mean_residual_deviance  10.3052   0.435185    10.3979       10.1599       10.8504       9.66564       10.452
mse                     10.3052   0.435185    10.3979       10.1599       10.8504       9.66564       10.452
r2                      0.820385  0.00959039  0.815415      0.825151      0.805846      0.827291      0.828222
residual_deviance       10.3052   0.435185    10.3979       10.1599       10.8504       9.66564       10.452
rmse                    3.20959   0.0680272   3.22458       3.18746       3.294         3.10896       3.23296
rmsle                   0.592079  0.024467    0.594752      0.594669      0.572747      0.568226      0.63

Scoring History: 
     timestamp            duration    number_of_trees    training_rmse       training_mae        training_deviance    validation_rmse     validation_mae      validation_deviance
---  -------------------  ----------  -----------------  ------------------  ------------------  -------------------  ------------------  ------------------  ---------------------
     2024-12-27 23:33:00  44.795 sec  0.0                nan                 nan                 nan                  nan                 nan                 nan
     2024-12-27 23:33:00  44.911 sec  1.0                5.290488688511849   3.5720561440503826  27.989270563271827   5.2741265860127955  3.50396310864096    27.816411245286982
     2024-12-27 23:33:00  45.005 sec  2.0                5.036426741507816   3.366124742064337   25.36559432257503    4.213337193125129   3.000037371246224   17.75221030297154
     2024-12-27 23:33:00  45.114 sec  3.0                5.062616157110254   3.4409416820425833  25.630082354233796   4.020098537748393   2.9604854834597676  16.161192253206767
     2024-12-27 23:33:00  45.236 sec  4.0                4.894872161019948   3.3472128992329715  23.95977347272809    3.8147203193849957  2.807537021347855   14.552091115128764
     2024-12-27 23:33:00  45.337 sec  5.0                4.809031589120416   3.270020821579136   23.126784825158026   3.6212302692558964  2.694296216758398   13.11330866297513
     2024-12-27 23:33:00  45.446 sec  6.0                4.7672724684741175  3.26400502261558    22.726886788671305   3.5

In [90]:
bagged_perf = rf_model.model_performance(test_data=test)
bagged_perf 

ModelMetricsRegression: drf
** Reported on test data. **

MSE: 9.639365979153151
RMSE: 3.104732835390696
MAE: 2.429011898675345
RMSLE: 0.5857100543387649
Mean Residual Deviance: 9.639365979153151

In [91]:
bagged_predictions = rf_model.predict(test)
bagged_predictions.head()

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict
21.0081
8.68144
12.2096
17.5071
15.5879
4.93371
10.7269
16.7793
2.53943
12.4879


### Stacking

In [92]:
# Gradient Boosting Machine
# One of the Base Models (Level 0 Models)
gbm_model = H2OGradientBoostingEstimator(
    ntrees=100, max_depth=5, learn_rate=0.1, seed=42, nfolds=5, keep_cross_validation_predictions=True
)
gbm_model.train(x=features, y=target, training_frame=train, validation_frame=val)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_model_python_1735308900165_64


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    100                100                         41143                  5            5            5             17            32            28.05

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 1.5053947798280132
RMSE: 1.2269453043343101
MAE: 0.948153670756943
RMSLE: NaN
Mean Residual Deviance: 1.5053947798280132

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 5.777318684637966
RMSE: 2.40360535126671
MAE: 1.7420051892417887
RMSLE: NaN
Mean Residual Deviance: 5.777318684637966

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 5.73388095690756
RMSE: 2.3945523500035577
MAE: 1.7482398225756408
RMSLE: NaN
Mean Residual Deviance: 5.73388095690756

Cross-Validation Metrics Summary: 
                        mean      sd         cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  --------  ---------  ------------  ------------  ------------  ------------  ------------
aic                     nan       0          nan           nan           nan           nan           nan
loglikelihood           nan       0          nan           nan           nan           nan           nan
mae                     1.74821   0.0612959  1.75784       1.7625        1.82435       1.65386       1.74247
mean_residual_deviance  5.7297    0.50494    5.81298       6.01881       6.33756       5.04033       5.43881
mse                     5.7297    0.50494    5.81298       6.01881       6.33756       5.04033       5.43881
r2                      0.900075  0.0101739  0.896807      0.896418      0.886598      0.909937      0.910614
residual_deviance       5.7297    0.50494    5.81298       6.01881       6.33756       5.04033       5.43881
rmse                    2.3918    0.106106   2.41101       2.45333       2.51745       2.24507       2.33213
rmsle                   nan       0          nan           nan           nan           nan           nan

Scoring History: 
     timestamp            duration    number_of_trees    training_rmse       training_mae        training_deviance    validation_rmse     validation_mae      validation_deviance
---  -------------------  ----------  -----------------  ------------------  ------------------  -------------------  ------------------  ------------------  ---------------------
     2024-12-27 23:33:22  10.409 sec  0.0                7.575565475958797   6.8359982780259285  57.38919228053883    7.5782555687386     6.838680576105014   57.4299574651176
     2024-12-27 23:33:22  10.445 sec  1.0                7.129297524751325   6.412002301080735   50.82688319642538    7.175981105140421   6.4549019572524875  51.494704821332334
     2024-12-27 23:33:22  10.466 sec  2.0                6.712264271709949   6.011700986047043   45.05449165327389    6.787415867060614   6.076728150346542   46.069014152426185
     2024-12-27 23:33:22  10.488 sec  3.0                6.360409729178818   5.6791361702910095  40.454811923032565   6.473190672403526   5.769404438110226   41.90219748129202
     2024-12-27 23:33:22  10.511 sec  4.0                6.017854012590231   5.344429345592996   36.21456691684835    6.163288035057683   5.450715167735381   37.98611940308519
     2024-12-27 23:33:22  10.533 sec  5.0                5.7480943215556275  5.084057787832137   33.040588329500046   5.934774356150227   5.226618923887742   35.22154665841834
     2024-12-27 23:33:22  10.557 sec  6.0                5.485003865951723   4.828666671288933   30.085267409505338   5.704024293100566   4.991729012586336   32

In [93]:
# Generalized Linear Model
# One of the Base Models (Level 0 Models)
glm_model = H2OGeneralizedLinearEstimator(family="gaussian", nfolds=5, keep_cross_validation_predictions=True)
glm_model.train(x=features, y=target, training_frame=train, validation_frame=val)

glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
,gaussian,identity,"Elastic Net (alpha = 0.5, lambda = 0.00595 )",50,48,1,py_36_sid_a7f3
,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
aic,2608.5618,82.566505,2502.5393,2556.1562,2718.6091,2633.4846,2632.0188
loglikelihood,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mae,0.5426220,0.0143119,0.5349909,0.5449228,0.5666661,0.5343979,0.5321322
mean_residual_deviance,0.5827644,0.0331848,0.5601175,0.5550763,0.638499,0.5785654,0.5815641
mse,0.5827644,0.0331848,0.5601175,0.5550763,0.638499,0.5785654,0.5815641
null_deviance,62701.91,1357.659,60384.79,63390.65,63142.43,63857.074,62734.6
r2,0.9898315,0.0005782,0.9901137,0.9904369,0.9888971,0.9899467,0.989763
residual_deviance,636.424,40.76543,595.4049,606.1433,699.7949,639.8933,640.8836


In [94]:
# Combine Base Models into a Stacked Ensemble
stacked_ensemble = H2OStackedEnsembleEstimator(
    base_models=[rf_model.model_id, gbm_model.model_id, glm_model.model_id],  # Base model IDs
    metalearner_algorithm="glm",
    seed=42
)

# Train the stacked ensemble
stacked_ensemble.train(x=features, y=target, training_frame=train, validation_frame=val)

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%


Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_model_python_1735308900165_66

No summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.468522755448609
RMSE: 0.6844872208073786
MAE: 0.49821320077046144
RMSLE: NaN
Mean Residual Deviance: 0.468522755448609
R^2: 0.9918360454846916
Null degrees of freedom: 5458
Residual degrees of freedom: 5455
Null deviance: 313287.60065946146
Residual deviance: 2557.6657219939566
AIC: 11363.117558521788

ModelMetricsRegressionGLM: stackedensemble
** Reported on validation data. **

MSE: 0.5541738530971202
RMSE: 0.7444285412966917
MAE: 0.5527639928199307
RMSLE: NaN
Mean Residual Deviance: 0.5541738530971202
R^2: 0.9903504354071277
Null degrees of freedom: 1169
Residual degrees of freedom: 1166
Null deviance: 67193.0502341876
Residual deviance: 648.3834081236306
AIC: 2639.692279863798

[tips]
Use `model.explain()` to inspect the model.
--
Use `h2o.display.toggle_user_tips()` to switch on/off this section.

In [95]:
stacked_perf = stacked_ensemble.model_performance(test)
stacked_perf

ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 0.5697789470522773
RMSE: 0.7548370334398526
MAE: 0.5504053162843511
RMSLE: NaN
Mean Residual Deviance: 0.5697789470522773
R^2: 0.9900681049442854
Null degrees of freedom: 1169
Residual degrees of freedom: 1166
Null deviance: 67121.28489254994
Residual deviance: 666.6413680511644
AIC: 2672.1832051924202

In [96]:
stacked_predictions = stacked_ensemble.predict(test)
stacked_predictions.head()

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict
24.0241
6.88873
15.6187
17.4896
16.7289
4.26075
11.2072
18.3169
0.731514
17.9857


### Voting

In [97]:
# Combine Base Models into a Voted Ensemble
voted_ensemble = H2OStackedEnsembleEstimator(
    base_models=[rf_model.model_id, gbm_model.model_id, glm_model.model_id],  # Base model IDs
    seed=42
)

# Train the stacked ensemble
voted_ensemble.train(x=features, y=target, training_frame=train, validation_frame=val)

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),2/3
# GBM base models (used / total),1/1
# DRF base models (used / total),0/1
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,AUTO
Metalearner nfolds,0
Metalearner fold_column,None
Custom metalearner hyperparameters,None


In [98]:
voted_perf = voted_ensemble.model_performance(test)
voted_perf

ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 0.5515571056454208
RMSE: 0.7426689071486842
MAE: 0.5361208239614378
RMSLE: NaN
Mean Residual Deviance: 0.5515571056454208
R^2: 0.9903857323636751
Null degrees of freedom: 1169
Residual degrees of freedom: 1167
Null deviance: 67121.28489254994
Residual deviance: 645.3218136051423
AIC: 2632.1545851655364

In [99]:
voted_predictions = voted_ensemble.predict(test)
voted_predictions.head()

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict
24.0822
6.88326
15.8439
17.386
16.7153
4.26303
11.3443
18.2658
0.751958
18.024


### Summary
Based on the performance metrics provided for **Bagging**, **Stacking**, and **Voting** models, we can compare the models using several metrics:

### **Key Performance Metrics**:
1. **Mean Squared Error (MSE)**: Lower MSE indicates better performance.
2. **Root Mean Squared Error (RMSE)**: Lower RMSE indicates better performance.
3. **Mean Absolute Error (MAE)**: Lower MAE indicates better performance.
4. **R² (R-squared)**: Higher R² indicates better model performance, as it shows the proportion of variance explained by the model.
5. **AIC**: Lower AIC indicates a better model (penalizes models for overfitting).
6. **Residual Deviance**: Lower residual deviance indicates better model performance.
   
### **Comparison of Models**:

#### **Bagged Model (DRF)**:
- **MSE**: 9.64
- **RMSE**: 3.10
- **MAE**: 2.43
- **R²**: Not provided
- **Residual Deviance**: 9.64

#### **Stacked Ensemble (GLM)**:
- **MSE**: 0.57
- **RMSE**: 0.75
- **MAE**: 0.55
- **R²**: 0.99
- **Residual Deviance**: 0.57
- **AIC**: 2671.23

#### **Voted Ensemble (GLM)**:
- **MSE**: 0.55
- **RMSE**: 0.74
- **MAE**: 0.54
- **R²**: 0.99
- **Residual Deviance**: 0.55
- **AIC**: 2632.42

### **Analysis**:
- **MSE, RMSE, and MAE**: The **Voted Ensemble** has the lowest MSE (0.55), RMSE (0.74), and MAE (0.54), which are all indicators of the best overall performance in terms of prediction error.
  
- **R²**: Both the **Stacked** and **Voted** models have very high R² values (~0.99), indicating that they explain almost all of the variance in the data, suggesting that both these models have excellent predictive power. The **Voted Ensemble** has a slightly higher R² (0.99 vs. 0.98 for Stacked), but the difference is minimal.

- **Residual Deviance**: Both **Stacked** and **Voted** models have very low residual deviance (around 0.55–0.57), which means the models are well-calibrated.

- **AIC**: The **Voted Ensemble** has the lowest AIC (2632.42), indicating that it has a better balance of model fit and complexity compared to the **Stacked Ensemble** (2671.23). Lower AIC values are preferable.

### **Conclusion**:
- The **Voted Ensemble** model shows the best performance based on most of the metrics, including **MSE**, **RMSE**, **MAE**, **R²**, and **AIC**.
- The **Stacked Ensemble** also performs very well, but it has slightly higher values for **MSE**, **RMSE**, and **AIC** than the **Voted Ensemble**.
- The **Bagged Model (DRF)** performs the worst overall with higher error metrics (MSE, RMSE, MAE), and no R² value is provided for comparison.

Therefore, the **Voted Ensemble** is the best-performing model among the three.